# Online Retail Dataset Analysis Report

## Dataset Loading and Initial Overview

To start the analysis, we loaded the dataset from the file 'Online Retail.xlsx' and created a copy named `df_original`. Here are the initial steps taken:

In [ ]:
!pip install --upgrade pip
!pip install --upgrade seaborn

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.ensemble import IsolationForest

In [ ]:
# Load the dataset
file_path = 'Online Retail.xlsx'
df_original = pd.read_excel(file_path, sheet_name='Online Retail')
df = df_original.copy()

In [ ]:
# Initial exploration
#df.info()

# Handling missing values
df.isnull().sum()

# We notice that only details from the description or customer number are missing, not from the 
# stock number. Additionally, we see that for the rest, there are no missing values, therefore 
# it is not even necessary to apply dropna.

Upon loading the dataset, we observed that missing values were primarily confined to the 'Description' and 'CustomerID' columns, while the 'StockCode' column had no missing values. 

```python
#Handling missing values
df.isnull().sum()
```

Given that other columns showed no missing values, there was no need to apply further data cleaning techniques for missing values (`dropna()`). Columns such as 'InvoiceNo', 'Description', and 'CustomerID' are dropped from the DataFrame (`df`) using `.drop(columns=columns_to_drop)`. These columns are considered unnecessary for subsequent analysis tasks.

#### Country Column Adjustment
- The value 'United Kingdom' in the 'Country' column is replaced with 'UK' using `.replace()`. This standardizes the country names for consistency in analysis and reporting.

These data preparation steps ensure that the DataFrame (`df`) is streamlined and suitable for further exploratory data analysis or modeling tasks in the context of online retail analysis. Note also that the column 'StockCode' includes both strings and integers, and it would actually be necessary to standardize all of them to strings.

In [ ]:
# Remove unnecessary columns for analysis
columns_to_drop = ['InvoiceNo','Description', 'CustomerID']
df = df.drop(columns=columns_to_drop)
# Filter rows where 'Quantity' or 'UnitPrice' is zero or non-sense for articles
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] >= 0.01)]
df['Country'].replace('United Kingdom', 'UK', inplace=True)

In [ ]:
basic_stats = df.describe()
basic_stats

# There is a lot of standard deviation in quantities and prices
# Great amount of outliers

### Identifying Outliers Using Interquartile Range (IQR) 


- **We give a theorical computation. Later, we will state the concrete code**



Based on the statistical summary provided:

- **Quantity:**
  - Q1 (25th percentile): 1.00
  - Q3 (75th percentile): 10.00
  - IQR (Interquartile Range): $10.00 - 1.00 = 9.00$

  To identify outliers:
  - Values below $Q1 - 1.5 \times IQR$: $1.00 - 1.5 \times 9.00 = -12.50$ (not applicable since Quantity cannot be negative)
  - Values above $Q3 + 1.5 \times IQR$: $10.00 + 1.5 \times 9.00 = 24.50$

  
  
  ######  Therefore, any Quantity value above 24.50 could be considered statistically as an outlier.



- **UnitPrice:**
  - Q1 (25th percentile): 1.25
  - Q3 (75th percentile): 4.13
  - IQR (Interquartile Range): $4.13 - 1.25 = 2.88$

  To identify outliers:
  - Values below $Q1 - 1.5 \times IQR$: $1.25 - 1.5 \times 2.88 = -2.875$ (not applicable since UnitPrice cannot be negative)
  - Values above $Q3 + 1.5 \times IQR$: $4.13 + 1.5 \times 2.88 = 8.88$



######  Therefore, any UnitPrice value above 8.88 could be considered statistically as an outlier.

- These thresholds are based on the interquartile range (IQR) method for outlier detection, providing a guideline to identify potential outliers in the 'Quantity' and 'UnitPrice' columns. Adjustments to these thresholds may be necessary depending on specific business context and data characteristics.

In [ ]:
# Calculate the interquartile range (IQR) for Quantity
Q1 = df['Quantity'].quantile(0.25)
Q3 = df['Quantity'].quantile(0.75)
IQR = Q3 - Q1

# Filter outliers in Quantity
filtered_df_IQR = df[(df['Quantity'] >= Q1 - 1.5 * IQR) & (df['Quantity'] <= Q3 + 1.5 * IQR)]

# Calculate the interquartile range (IQR) for UnitPrice
Q1_price = df['UnitPrice'].quantile(0.25)
Q3_price = df['UnitPrice'].quantile(0.75)
IQR_price = Q3_price - Q1_price

# Filter outliers in UnitPrice
filtered_df_IQR = filtered_df_IQR[(filtered_df_IQR['UnitPrice'] >= Q1_price - 1.5 * IQR_price) & 
                          (filtered_df_IQR['UnitPrice'] <= Q3_price + 1.5 * IQR_price)]

stats_IQR = filtered_df_IQR.describe()
stats_IQR

## Training Isolation Forest on IQR Filtered Data

We have previously applied multiple outlier detection methods to our dataset, including the Interquartile Range (IQR), Isolation Forest, Z-Score, and Tukey's method. After reviewing the descriptive statistics generated by these methods, we observed that the IQR method provided the most reliable and representative data.

To further refine our dataset and ensure robustness in outlier detection, we will now train the Isolation Forest model on the data that has already been filtered using the IQR method. This approach aims to enhance our outlier detection by leveraging the strengths of both statistical and machine learning techniques.

Here’s how we will proceed:

1. **Create a copy of the IQR-filtered DataFrame**:
   - This step ensures that our original IQR-filtered data remains intact for future reference.

2. **Train the Isolation Forest model**:
   - We will configure the Isolation Forest with a contamination parameter of 5% to identify potential outliers.

3. **Fit the model and predict outliers**:
   - The model will be trained on the 'Quantity' and 'UnitPrice' columns of the IQR-filtered data. It will label each entry as an outlier or inlier.

4. **Filter out identified outliers**:
   - Entries identified as outliers by the Isolation Forest will be excluded from our dataset, providing a cleaner and more accurate representation of our data.

5. **Analyze the refined dataset**:
   - We will generate descriptive statistics for the refined dataset to validate the effectiveness of our combined outlier detection approach.

By combining the statistical rigor of the IQR method with the machine learning capabilities of the Isolation Forest, we aim to achieve a more robust and accurate dataset for subsequent analysis and decision-making.

In [ ]:
# Create a copy of the DataFrame filtered by the IQR method
df_IQR_fit = filtered_df_IQR.copy()

# Create the Isolation Forest model
clf_over_IQR = IsolationForest(contamination=0.05)  # Specify contamination parameter

# Fit the model and predict outliers for 'Quantity' and 'UnitPrice'
df_IQR_fit['Outlier_IF'] = clf_over_IQR.fit_predict(df_IQR_fit[['Quantity', 'UnitPrice']])

# Filter the DataFrame to exclude outliers (where 'Outlier_IF' == 1)
filtered_df_isolated_forest_over_IQR = df_IQR_fit[df_IQR_fit['Outlier_IF'] == 1].drop(columns=['Outlier_IF'])
df_cleaned = filtered_df_isolated_forest_over_IQR.copy()

# Calculate the descriptive statistics of the filtered DataFrame
stats_isolated_forest_over_IQR = filtered_df_isolated_forest_over_IQR.describe()

stats_isolated_forest_over_IQR

After applying the Isolation Forest algorithm to detect and remove outliers from the dataset, the statistical summary of 'Quantity' and 'UnitPrice' shows improvements in their standard deviations.

The standard deviation (std) measures the variability or spread of data points around the mean. A lower standard deviation post Isolation Forest indicates that the data points are more tightly clustered around the mean, suggesting reduced variability and improved data consistency. This improvement enhances the reliability of statistical analyses and insights derived from the dataset.

In [ ]:
# Histogram of Quantity
plt.figure(figsize=(10, 6))
sns.histplot(df_cleaned['Quantity'], bins=50, kde=True)
plt.title('Distribution of Quantity')
plt.show()

In [ ]:
# Histogram of UnitPrice
plt.figure(figsize=(10, 6))
sns.histplot(df_cleaned['UnitPrice'], bins=50, kde=True)
plt.title('Distribution of UnitPrice')
plt.show()

In [ ]:
# Scatter plot of Quantity vs UnitPrice
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Quantity', y='UnitPrice', data=df_cleaned)
plt.title('Quantity vs UnitPrice')
plt.show()

In [ ]:
df_cleaned['InvoiceDate'] = pd.to_datetime(df_cleaned['InvoiceDate'])

df_cleaned['Month'] = df_cleaned['InvoiceDate'].dt.month
df_cleaned['DayOfWeek'] = df_cleaned['InvoiceDate'].dt.dayofweek

monthly_sales = df_cleaned.groupby('Month')['Quantity'].sum()

plt.figure(figsize=(10, 6))
monthly_sales.plot(kind='bar')
plt.title('Sales by Month')
plt.xlabel('Month')
plt.ylabel('Total Quantity Sold')
plt.show()

daily_sales = df_cleaned.groupby('DayOfWeek')['Quantity'].sum()

plt.figure(figsize=(10, 6))
daily_sales.plot(kind='bar')
plt.title('Sales by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Total Quantity Sold')
plt.show()

In [ ]:
# Paso 6: Productos y países más vendidos
# Productos más vendidos
top_products = df_cleaned.groupby('StockCode')['Quantity'].sum().sort_values(ascending=False).head(10)

plt.figure(figsize=(10, 6))
top_products.plot(kind='bar')
plt.title('Top-Selling Products')
plt.xlabel('StockCode')
plt.ylabel('Total Quantity Sold')
plt.show()

# Crear el gráfico de pastel
plt.figure(figsize=(10, 6))
top_products.plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.title('Top-Selling Products')
plt.ylabel('')  # Eliminar el nombre del eje y
plt.show()

In [ ]:
stock_codes_top_products = top_products.index.tolist()
filtered_df = df_original[['Description', 'StockCode']]
filtered_df = filtered_df[filtered_df['StockCode'].isin(stock_codes_top_products)]
filtered_df=filtered_df.drop_duplicates(subset='StockCode')
filtered_df['Occurrences'] = filtered_df['StockCode'].apply(lambda x: count_stockcode_occurrences(top_products, x))
filtered_df = filtered_df.sort_values(by='Occurrences',ascending = False).reset_index(drop=True)
filtered_df

### Insights from Sales Analysis:

- **November Sales Peak:**
  November emerges as the month with the highest sales volume, indicating robust customer activity and transactional throughput during this period.

- **Thursday Sales Dominance:**
  Thursdays stand out as the top-performing day of the week in terms of sales, consistently generating the highest sales volume compared to other weekdays.

- **Top-Selling Products:**
  The products with StockCodes 84879, 85099B, and 85123A are identified as the top-sellers based on their consistently high sales quantities. These items are particularly popular among customers, highlighting their significant contribution to overall sales.
  
### Top 10 Selling Products with Descriptions:

| Index | Description                              | StockCode |
|-------|------------------------------------------|-----------|
| 0     | ASSORTED COLOUR BIRD ORNAMENT             | 84879     |
| 1     | JUMBO BAG RED RETROSPOT                   | 85099B    |
| 2     | WHITE HANGING HEART T-LIGHT HOLDER        | 85123A    |
| 3     | LUNCH BAG RED RETROSPOT                   | 20725     |
| 4     | JAM MAKING SET PRINTED                    | 22961     |
| 5     | mailout                                  | 23203     |
| 6     | HEART OF WICKER SMALL                     | 22469     |
| 7     | SMALL POPCORN HOLDER                      | 22197     |
| 8     | SET OF 4 PANTRY JELLY MOULDS              | 22993     |
| 9     | LUNCH BAG SUKI DESIGN                     | 22383     |


These insights are pivotal for strategic decision-making, enabling the optimization of marketing efforts, inventory management, and customer engagement strategies to capitalize on peak sales periods and popular products.

In [ ]:
# Group by country and sum the quantities
country_sales = df_cleaned.groupby('Country')['Quantity'].sum().sort_values(ascending=False).head(10)

# Plotting histogram of quantity sold by country
plt.figure(figsize=(12, 6))
sns.barplot(x=country_sales.values, y=country_sales.index, palette='viridis')
plt.title('Top 10 Countries by Quantity Sold')
plt.xlabel('Total Quantity Sold')
plt.ylabel('Country')
plt.show()

In [ ]:
# Group by country and sum the quantities, excluding UK
country_sales = df_cleaned[df_cleaned['Country'] != 'UK'].groupby('Country')['Quantity'].sum().sort_values(ascending=False).head(10)

# Plotting histogram of quantity sold by country excluding UK
plt.figure(figsize=(12, 6))
sns.barplot(x=country_sales.values, y=country_sales.index, palette='viridis')
plt.title('Top 10 Countries by Quantity Sold (Excluding UK)')
plt.xlabel('Total Quantity Sold')
plt.ylabel('Country')
plt.show()

In [ ]:
# Filter the DataFrame to include only UK data
uk_sales = df_cleaned[df_cleaned['Country'] == 'UK']

# Get the top-selling products in the UK
top_products_uk = uk_sales.groupby('StockCode')['Quantity'].sum().sort_values(ascending=False).head(10)
top_products_uk

# Plot the histogram of top-selling products in the UK
plt.figure(figsize=(12, 6))
sns.barplot(x=top_products_uk.index, y=top_products_uk.values, palette='muted')
plt.title('Top 10 Selling Products in the UK')
plt.xlabel('StockCode')
plt.ylabel('Total Quantity Sold')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Filter the DataFrame to include only Germany data
germany_sales = df_cleaned[df_cleaned['Country'] == 'Germany']

# Get the top-selling products in Germany
top_products_germany = germany_sales.groupby('StockCode')['Quantity'].sum().sort_values(ascending=False).head(10)
top_products_germany

# Plot the histogram of top-selling products in Germany
plt.figure(figsize=(12, 6))
sns.barplot(x=top_products_germany.index, y=top_products_germany.values, palette='muted')
plt.title('Top 10 Selling Products in Germany')
plt.xlabel('StockCode')
plt.ylabel('Total Quantity Sold')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
# Filter the DataFrame to include only France data
france_sales = df_cleaned[df_cleaned['Country'] == 'France']

# Get the top-selling products in France
top_products_france = france_sales.groupby('StockCode')['Quantity'].sum().sort_values(ascending=False).head(10)
top_products_france

# Plot the histogram of top-selling products in France
plt.figure(figsize=(12, 6))
sns.barplot(x=top_products_france.index, y=top_products_france.values, palette='muted')
plt.title('Top 10 Selling Products in France')
plt.xlabel('StockCode')
plt.ylabel('Total Quantity Sold')
plt.xticks(rotation=45, ha='right')
plt.show()

In this dataset, the disproportionate size of the UK market compared to other countries means that the top-selling products in the UK have a significant impact on overall market trends. For example, while the top-selling products in Germany and France may differ entirely, the products that dominate in the UK heavily influence the top-selling items across the entire market.

This is primarily due to the UK market contributing a substantial portion of the total sales data. Therefore, products that are popular in the UK can skew the overall market trends, leading them to appear as top-selling items in the dataset analysis.

From a practical perspective, this underscores the importance of considering the influence of the UK market when analyzing or making decisions based on the top-selling products across all countries. Products consistently favored in the UK are likely to play a pivotal role in strategic business decisions and marketing strategies aimed at optimizing overall sales performance.